# [Title]

## 1. Introduction

### 1.1 Literature review of the compound (<500 words)

### 1.2 Literature review of the cell factory (<500 words)

## 2. Problem definition (<300 words)

## 3. *If Project category I:* Reconstruction of a new GSM for your cell factory host of interest (<1500 words)

or

## 3. *If Project category II:* Selection and assessment of existing GSM (<500 words)

As previously mentioned, the model used for this assignment was provided by Imam et al. (2015) as an improvement of the previous model iRC1080. After completing the model, the authors actually conducted several experiments to contrast the accuracy of the new model's predictions.

A collection of 306 different genotype-phenotype conditions, obtained from 81 different mutants, was listed, and iCre1335 was used to predict the phenotypes of these mutants. The model showed a sensitivity of 83%, and a specificity of 92%, overall achieving an area under the Receiver Operating Curve (ROC) of 0.92. This means that only in 17% of the cases, the expression of a phenotype was predicted incorrectly; and in 8% of the cases, the lack of expression of a phenotype was predicted incorrectly.

Authors further tested the model by comparing the predicted growth rates under different metabolic conditions and C, N and P uptake rates with experimental cultures, obtaining a good agreement ($R^{2}$ = 0.82) between experimental and predicted values. Moreover, the model actually was able to predict growth halt and lipid accumulation due to nitrogen starvation, but unfortunately, it was unable to correctly predict the effect of light intensity changes in growth rate. In this case, the authors had to manually tune the photon uptake rate to match predictions with experimental results.

Overall, the authors were fairly confident of the predictive capabilities of the model. In order to independently verify that statement, we conducted our own validation, by uploading the file to memote, to analyse the consistency of the model. The report obtained, which has been added to this project's repository, shows that the model was quite satisfactory. Almost the totality of the reactions are both mass and charge balanced, there are almost no disconnected metabolite (meaning that they are not used in any reaction) and most of the reactions are not able to carry unlimited amounts of flux. The main problems are the lack of stoichiometric consistency, meaning that it is not possible to assign to each metabolite a positive molecular mass without breaking mass conservation at some point in the model (this is probably caused by the few reactions that are not mass balanced); and the fact that all metabolites, reactions and genes do not have any reference (annotation) linking them to one major database.

In [ ]:
!memote report snapshot iCre1355_hetero_V2.xml

## 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project; <500 words for Category I project)

# Model upload and general information

Firstly, the model given by the authors was loaded into the script, but it creates problems during the reading, due to formating of the data (conflicts with SBML definitions).

In [4]:
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model

In [ ]:
hetero = read_sbml_model('iCre1355_hetero.xml')
auto = read_sbml_model('iCre1355_auto.xml')
mixo = read_sbml_model('iCre1355_mixo.xml')

# Source: https://github.com/baliga-lab/Chlamy_model_iCre1355.git

Therefore, in order to avoid constant warning messages that slow down computation, after the model is read for the first time, a new model is written with the correct format

In [ ]:
write_sbml_model(hetero, 'iCre1355_hetero_V2.xml')
write_sbml_model(auto, 'iCre1355_auto_V2.xml')
write_sbml_model(mixo, 'iCre1355_mixo_V2.xml')

In [5]:
hetero = read_sbml_model('iCre1355_hetero_V2.xml')
auto = read_sbml_model('iCre1355_auto_V2.xml')
mixo = read_sbml_model('iCre1355_mixo_V2.xml')

In [6]:
hetero.name = 'iCre1355_Hetero'
auto.name = 'iCre1355_Auto'
mixo.name = 'iCre1355_Mixo'

models = [auto, mixo, hetero]
bio_growth = [auto.reactions.Biomass_Chlamy_auto, mixo.reactions.Biomass_Chlamy_mixo, hetero.reactions.Biomass_Chlamy_hetero]

Let´s now take a first look at the models. Let´s begin by checking the number of reactions, metabolites and genes:

In [7]:
for model in models:
    print('============================\n', model.name, '\n----------------------------')
    print('{0:22}  {1}'.format('Number of metabolites:', len(model.metabolites)))
    print('{0:22}  {1}'.format('Number of reactions:', len(model.reactions)))
    print('{0:22}  {1}'.format('Number of genes:', len(model.genes)))
    print('\n')

 iCre1355_Auto 
----------------------------
Number of metabolites:  1845
Number of reactions:    2394
Number of genes:        1963


 iCre1355_Mixo 
----------------------------
Number of metabolites:  1845
Number of reactions:    2394
Number of genes:        1963


 iCre1355_Hetero 
----------------------------
Number of metabolites:  1845
Number of reactions:    2394
Number of genes:        1963




Now, let's analyze other useful information, like the number of compartments, the media composition and the available exchange reactions

In [8]:
# Medium

for model in models:
    print('============================\n', model.name, '\n----------------------------')
    for compound in model.medium:
        print('{0:15}  {1}'.format(compound, model.medium[compound]))
    print('\n')

 iCre1355_Auto 
----------------------------
EX_h_e           10.0
EX_h2o_e         10.0
EX_pi_e          10.0
EX_nh4_e         1.0
EX_so4_e         10.0
EX_fe2_e         10.0
EX_mg2_e         10.0
EX_na1_e         10.0
EX_photonVis_e   80.0
EX_o2_e          10.0
EX_co2_e         2.0


 iCre1355_Mixo 
----------------------------
EX_h_e           10.0
EX_h2o_e         10.0
EX_pi_e          10.0
EX_nh4_e         1.0
EX_so4_e         10.0
EX_fe2_e         10.0
EX_mg2_e         10.0
EX_na1_e         10.0
EX_photonVis_e   80.0
EX_o2_e          10.0
EX_co2_e         2.0
EX_ac_e          2.0


 iCre1355_Hetero 
----------------------------
EX_h_e           10.0
EX_h2o_e         10.0
EX_pi_e          10.0
EX_nh4_e         0.5
EX_so4_e         10.0
EX_fe2_e         10.0
EX_mg2_e         10.0
EX_na1_e         10.0
EX_o2_e          10.0
EX_ac_e          2.0




In [9]:
# Compartments

for model in models:
    print('============================\n', model.name, '\n----------------------------')
    for compartment in model.compartments:
        print('{0:2}  {1}'.format(compartment, model.compartments[compartment]))
    print('\n')

 iCre1355_Auto 
----------------------------
c   Cytosol
h   Chloroplast
m   Mitochondria
x   Glyoxysome
f   Flagellum
e   Extra-organism
n   Nucleus
g   Golgi Apparatus
s   Eyespot
u   Thylakoid Lumen
i   Inner Mitochondrial membrane space


 iCre1355_Mixo 
----------------------------
c   Cytosol
h   Chloroplast
m   Mitochondria
x   Glyoxysome
f   Flagellum
e   Extra-organism
n   Nucleus
g   Golgi Apparatus
s   Eyespot
u   Thylakoid Lumen
i   Inner Mitochondrial membrane space


 iCre1355_Hetero 
----------------------------
c   Cytosol
h   Chloroplast
m   Mitochondria
x   Glyoxysome
f   Flagellum
e   Extra-organism
n   Nucleus
g   Golgi Apparatus
s   Eyespot
u   Thylakoid Lumen
i   Inner Mitochondrial membrane space




In [10]:
# Available exchange reactions

for model in models:
    print('==========================================================================\n', model.name, '\n--------------------------------------------------------------------------')
    for exchange in model.exchanges:
        print('{0:38}  {1:15}  {2}'.format(exchange.name, exchange.id, exchange.reaction))
    print('\n')

 iCre1355_Auto 
--------------------------------------------------------------------------
H+ exchange                             EX_h_e           h_e <=> 
H2O exchange                            EX_h2o_e         h2o_e <=> 
Phosphate exchange                      EX_pi_e          pi_e <=> 
ammonia exchange                        EX_nh4_e         nh4_e <=> 
nitrate exchange                        EX_no3_e         no3_e --> 
sulfate exchange                        EX_so4_e         so4_e <=> 
Fe2+ exchange                           EX_fe2_e         fe2_e <=> 
Fe3+ exchange                           EX_fe3_e         fe3_e --> 
magnesium exchange                      EX_mg2_e         mg2_e <=> 
sodium exchange                         EX_na1_e         na1_e <=> 
photon emission                         EX_photonVis_e   photonVis_e <=> 
O2 exchange                             EX_o2_e          o2_e <=> 
CO2 exchange                            EX_co2_e         co2_e <=> 
HCO3 exchange          

# Theoretical maximum yields & phenotipic phase planes

Finally, let`s run the models and see the growth rate for each of them, as well as $H_{2}$ production rate when the model is set to maximize biomass growth and when the model is set to maximize $H_{2}$ productivity

In [ ]:
for model, reaction in zip(models, bio_growth):
    model.objective = reaction
    with model:
        print('======================================================================\n', model.name, '\n----------------------------------------------------------------------')
        print(model.objective, '\n')
        print('{0:30}  {1}'.format('Biomass maximum growth rate:', model.optimize().objective_value))
        print('{0:30}  {1}'.format('Hydrogen productivity:', model.optimize().fluxes.EX_h2_e), '\n')        
        model.objective = model.reactions.EX_h2_e
        print('......................................................................')
        print(model.objective, '\n')
        print('{0:30}  {1}'.format('Biomass growth rate:', model.optimize().fluxes.get(reaction.id)))
        print('{0:30}  {1}'.format('Hydrogen maximum productivity:', model.optimize().objective_value), '\n\n\n\n')

This data must be taken with cautious, however, as it can be seen in the phenotypic phase planes, for the maximum biomass growth there are several hydrogens productivities that allow it.

Lastly, we'll compute the phenotypic phase plane, to have an idea of the relation between the maximum production of hydrogen (EX_h2_e) and the biomass growth.

In [ ]:
from cameo import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter

plotter = PlotlyPlotter()

for model, reaction in zip(models,bio_growth):
    ppp = phenotypic_phase_plane(model,
                                 variables=[model.reactions.get_by_id(reaction.id)],
                                 objective=model.reactions.EX_h2_e)

    ppp.plot(plotter, height=400)

It is possible that results do not concorde with the ones shown in the previous code cell, in chich the model was optimize for maximum production of biomass and hydrogen. This happens because, as it can be seen in the PPPs, there are several different hydrogen productivities that allow the maximum growth rate. Therefore, each time the model is run, a different result might appear.

Nevertheless, the best nutrition model seems to be the mixotrophic: even though for the highest biomass growth rate the hydrogen production is lower than for the autotrophic nutrition, if we compare the hydrogen productivity at the maximum growth rate in autotrophic conditions (around 6) with the hydrogen production at the same conditions in mixotrophic nutrition (around 9), hydrogen production is higher for the mixotrophic model

Note: If the visualization does not work, opern jupyter as /tree, not as /lab (in the search bar)

# Pathway visualization

In order to have a better general idea of how the model works, a package like escher can be used. Escher helps visualizing the methabolic pathways, but it only contains models for yeast, E. coli and humans, not for all microorganisms.

Nevertheless, a S. cerevisiae map can be loaded with Saccharomyces reactions, so only the common pathways will be contain fluxes (reactions that are only present in Chlamydomomas will however not be shown in the model). Saccharomyces has been selected because both it and Chlamydomomas are eukariot microorganisms

In [ ]:
import escher
escher.list_available_maps()

In [ ]:
escher.Builder('iMM904.Central carbon metabolism',
               model=auto, highlight_missing=True,
               reaction_data=auto.optimize().fluxes.to_dict()).display_in_notebook()

In [ ]:
escher.Builder?

Unfortunately, this approach was less useful than expected, as most of the reactions present in the model (´iJO1366.Central metabolism´ is one of the most complete E. coli models) were silent, meaning that they are not present in the Chlamydomomas model.

# Definition of the production pathways

Prior to any modification to the model, let´s look for the reactions that produce the desired compound:

In [ ]:
name = []
ID = []

for metabolite in mixo.metabolites.query('H2', 'name'):
    
    print(metabolite, '\t', metabolite.name)    
    if metabolite.name == 'H2':
        name.append(metabolite.name)
        ID.append(metabolite.id)
        
print(name, ID)

In [ ]:
reactions = []

for element in ID:
    compound = mixo.metabolites.get_by_id(element)
    for react in compound.reactions:
        reactions.append(react.id)

for element in reactions:
    print(mixo.reactions.get_by_id(element))

We can see that the reactions that actually produce hydrogen are HYDA, HYDAh and HYDAm, the other ones just reflect transport of hydrogen between different compartments. Both HYDA, HYDAh and HYDAm are in fact the same reaction, the reason why it is splitted into three is because it can take place in three different compartments. If we take one of the reactions and we look for the catalyzing enzyme, we could see that the microorganisms actually uses FeFe-hydrogenase, which was stated in the introduction as the most active enzyme for production of hydrogen.

In [ ]:
hydrogen_production = [mixo.reactions.HYDA, mixo.reactions.HYDAm, mixo.reactions.HYDAh]
for reaction in hydrogen_production:
    print(mixo.reactions.get_by_id(reaction.id).id, ': ', mixo.reactions.get_by_id(reaction.id).name)

Let`s see in which compartment the enzyme is more active, and thus, produces the highest amounts of $H_{2}$, while producing the highest amount of biomass

In [ ]:
for model in models:
    with model:
        print('======================================================================\n', model.name, '\n----------------------------------------------------------------------')
        print(model.objective, '\n')
        print('{0:50}  {1}'.format('Biomass maximum growth rate:', model.optimize().objective_value))
        print('{0:50}  {1}'.format('Hydrogen productivity in cytoplasm:', model.optimize().fluxes.HYDA))
        print('{0:50}  {1}'.format('Hydrogen productivity in chloroplast:', model.optimize().fluxes.HYDAh))
        print('{0:50}  {1}'.format('Hydrogen productivity in mitochrondia:', model.optimize().fluxes.HYDAm))
        print('{0:50}  {1}'.format('Hydrogen exchange:', model.optimize().fluxes.EX_h2_e))

# Influence of the media on Hydrogen production

Another factor that can have a potential impact on hydrogen production is the media composition. We'll analyze several scenarios in the following code:

In [ ]:
sulfur_uptake_reactions = []
nitrogen_uptake_reactions = []
phosphorous_uptake_reactions = []
carbon_uptake_reactions = []
new_growth_carbon = []
new_growth_nitrogen = []
new_growth_sulfur = []
new_growth_phosphorous = []

carbon_uptake_reactions = []
for reaction in mixo.exchanges:
    for element in reaction.metabolites:
        if 'S' in element.elements:
            sulfur_uptake_reactions.append(reaction.id)
        if 'N' in element.elements:
            nitrogen_uptake_reactions.append(reaction.id)
        if 'P' in element.elements:
            phosphorous_uptake_reactions.append(reaction.id)
        if 'C' in element.elements:
            carbon_uptake_reactions.append(reaction.id)

carbon_uptake_reactions.remove('EX_co2_e')
        

medium = mixo.medium
with mixo:
    print('============================\nOrganic carbon sources\n----------------------------')
    for element in carbon_uptake_reactions:
        medium['EX_ac_e'] = 0
        medium[element] = 12.0
        mixo.medium = medium
        new_growth_carbon.append(mixo.slim_optimize())
        print('{0:15}:  {1}.    Biomass growth: {2}'.format(element, medium[element], new_growth_carbon[-1]))
        medium[element] = 0
    best_carbon_source = carbon_uptake_reactions[new_growth_carbon.index(max(new_growth_carbon))]
    print('Best carbon source: ', best_carbon_source)
    medium[best_carbon_source] = 12.0    
    
    print('\n\n============================\nSulfur sources\n----------------------------')
    for element in sulfur_uptake_reactions:
        medium['EX_so4_e'] = 0
        medium[element] = 12.0
        mixo.medium = medium
        new_growth_sulfur.append(mixo.slim_optimize())
        print('{0:15}:  {1}.    Biomass growth: {2}'.format(element, medium[element], new_growth_sulfur[-1]))
        medium[element] = 0
    best_sulfur_source = sulfur_uptake_reactions[new_growth_sulfur.index(max(new_growth_sulfur))]
    print('Best sulfur source: ', best_sulfur_source)
    medium[best_sulfur_source] = 12.0 
        
    print('\n\n============================\nNitrogen sources\n----------------------------')
    for element in nitrogen_uptake_reactions:
        medium['EX_nh4_e'] = 0
        medium[element] = 12.0
        mixo.medium = medium
        new_growth_nitrogen.append(mixo.slim_optimize())
        print('{0:15}:  {1}.    Biomass growth: {2}'.format(element, medium[element], new_growth_nitrogen[-1]))
        medium[element] = 0
    best_nitrogen_source = nitrogen_uptake_reactions[new_growth_nitrogen.index(max(new_growth_nitrogen))]
    print('Best nitrogen source: ', best_nitrogen_source)
    medium[best_nitrogen_source] = 12.0 
    
    print('\n\n============================\nPhosphorous sources\n----------------------------')
    for element in phosphorous_uptake_reactions:
        medium['EX_pi_e'] = 0
        medium[element] = 12.0
        mixo.medium = medium
        new_growth_phosphorous.append(mixo.slim_optimize())
        print('{0:15}:  {1}.    Biomass growth: {2}'.format(element, medium[element], new_growth_phosphorous[-1]))
        medium[element] = 0
    best_phosphorous_source = phosphorous_uptake_reactions[new_growth_phosphorous.index(max(new_growth_phosphorous))]
    print('Best phosphorous source: ', best_phosphorous_source)
    medium[best_phosphorous_source] = 12.0 

It is curious, however, the fact that the authors decided not to include glucose as a possible carbon source

The best media composition would have, then, (LIST ALL THE COMPOUNDS). This results are expected, as both glutamine and ornithine are aminoacids, meaning that they act at the same time as carbon and as nitrogen sources. Regarding nicotinamide mononucleotide, it is a complex vitamin that further acts as C and N source. Nevertheless, the aim of this analysis was to find the best media composition, without taking into account economic parameters (phosphate will probably be a much cheaper P source than Nicotinamide mononucleotide). Further, this analysis has been performed linearly, meaning that the yield achieved by a compound might be influenced by the previous choices of "best compounds". An analysis that could take into account correlations between compounds would be much more accurate and useful, though this analysis is enough to give an idea of the preferences of the microorganism.

Moreover, it is also important to check which effect might have the change in media composition in hydrogen production:

In [ ]:
with mixo:
    new_media = mixo.medium
    new_media["EX_ac_e"] = 0
    new_media["EX_nh4_e"] = 0
    new_media["EX_pi_e"] = 0
    new_media[best_carbon_source] = 12
    new_media[best_sulfur_source] = 12
    new_media[best_nitrogen_source] = 12
    new_media[best_phosphorous_source] = 12
    mixo.medium = new_media
    
    ppp_new_media = phenotypic_phase_plane(model,
                                variables=[model.reactions.get_by_id(reaction.id)],
                                objective=model.reactions.EX_h2_e)

    ppp_new_media.plot(plotter, height=400)

# Gene KnockOut strategies

Let's try now to enhance the production of hydrogen by OptKnock first:

In [ ]:
from cobra import Reaction, Metabolite
from cameo.strain_design.heuristic.evolutionary_based import OptGene
from cameo.strain_design.deterministic.linear_programming import OptKnock

Hydrogen_producer_reactions = ['HYDA', 'HYDAm', 'HYDAh']

for reaction in Hydrogen_producer_reactions
    with mixo:
        optknock = OptKnock(mixo, fraction_of_optimum=0.1)
        result = optknock.run(max_knockouts=2, target=reaction, biomass="Biomass_Chlamy_mixo")

Running OptGene in this microorganism would be pointless, as the model already produces hydrogen, and OptGene is used to force a model to produce the metabolite of interest when it does not do it naturally:

In [ ]:
with model:
    medium = model.medium
    medium['EX_lac_D_e'] = 100
    model.medium = medium
    model.objective = model.reactions.ALATLm
    optgene = OptGene(model)
    result = optgene.run(target=model.reactions.ALATLm, 
                         biomass=model.reactions.Biomass_Chlamy_hetero,
                         substrate=model.metabolites.lac_D_e,
                         max_evaluations=20, population_size=20, max_knockouts=5,
                         plot=False, growth_coupled=True)

In [ ]:
result

# Flux Scanning based on Enforced Objective Flux

After performing the FSEOF, it could be seen that when hydrogen production is forced to take higher values, sulfur metabolism also starts to increase: the flux of the reaction   increases, and the absolute value of the flux of reaction SO4t increases proportionally, excreting sulfur to the media (because it has a negative sign), and preventing it from accumulating within the cell.

The hypothesis is that this sulfur metabolism might be related with hydrogen production. Hence, this metabolism will be overexpressed, and the effect it has on hydrogen productivity will be checked.

In [ ]:
with mixo:
    print('Wild type growth rate: ', mixo.slim_optimize())
    ppp_wild = phenotypic_phase_plane(mixo,
                                 variables=[mixo.reactions.Biomass_Chlamy_mixo],
                                 objective=mixo.reactions.EX_h2_e)
    ppp_wild.plot(plotter, height=400)
    
    mixo.reactions.SO4NA1t.lower_bound = mixo.optimize().fluxes.SO4NA1t * 10
    print('Modified type growth rate: ', mixo.slim_optimize())
    ppp_mod = phenotypic_phase_plane(mixo,
                                 variables=[mixo.reactions.Biomass_Chlamy_mixo],
                                 objective=mixo.reactions.EX_h2_e)
    ppp_mod.plot(plotter, height=400)

Unfortunately, results are the same for both situation, disproving the hypothesis.

# Simulation of batch cultivation with dFBA

Taking into account all of the previous results, we'll now try to simulate the model with a dynamic FBA approach:

NOTE: only a Python 3 kernel is able to import the dfba module, Python3.6 kernel

In [ ]:
from os.path import dirname, join, pardir

from cobra.io import read_sbml_model

from dfba import DfbaModel, ExchangeFlux, KineticVariable

In [ ]:
%%capture --no-display
fba_model = mixo
fba_model.solver = "glpk"
dfba_model = DfbaModel(fba_model)

In [ ]:
X = KineticVariable("Biomass")
Gln = KineticVariable("L-glutamine")
Sulf = KineticVariable("Sulfate")
Orn = KineticVariable("Ornithin")
Nmn = KineticVariable("Nicotinamide mononucleotide")

dfba_model.add_kinetic_variables([X, Gln, Sulf, Orn, Oxy, Nmn, Hydro])

Gases have not been added to the model, because in a batch fermentation they are either fed continuosly in the reactor ($O_{2}$, $CO_{2}$) or they can leave the broth and not accumulate ($H_{2}$))

In [ ]:
mu = ExchangeFlux("Biomass_Chlamy_mixo")
v_C = ExchangeFlux("EX_gln_L_e")
v_S = ExchangeFlux("EX_so4_e")
v_N = ExchangeFlux("EX_orn_e")
v_P = ExchangeFlux("EX_nmn_e")

dfba_model.add_exchange_fluxes([mu, v_C, v_S, v_N, v_P, v_H])

In [ ]:
dfba_model.add_rhs_expression("Biomass", mu * X)
dfba_model.add_rhs_expression("L-glutamine", v_C * 146.14/1000 * X)
dfba_model.add_rhs_expression("Sulfate", v_S * 96.06/1000 * X)
dfba_model.add_rhs_expression("Ornithin", v_N * 132.16/1000 * X)
dfba_model.add_rhs_expression("Nicotinamide mononucleotide", v_P * 334.221/1000 * X)

## 5. Discussion (<500 words)

## 6. Conclusion (<200 words)

## References

What has to be done:
    
    Create exchange reactions for the lipids (otherwise they would accumulate, breaking the assumption of steady state)
    
    Reduce biomass growth in order to allow carbon to be diverted into lipid production pathways (otherwise it would not be driven towards this pathways, as it would be a lost of  C for growth) and to simulate N shortage (even though that would trigger a lot of genome expression / silention, as that is a stres condition in which cells are struggling to keep alive)
    
    Set the objecive function to the sum of the exchange reactions of lipids (optimize the total sum)
    
    Use escher with the map of E.coli, but with the reactions and fluxes of C.reinhardtii (some reactions wont be present in escher, others present in escher will have no flux at all)